In [ ]:
from grand.grandlib_classes.grandlib_classes import *
import grand.dataio.root_trees as rt
import numpy as np
import matplotlib.pyplot as plt
# import glob
from scipy.fft import rfftfreq, rfft, irfft
from grand import ECEF, Geodetic, GRANDCS, LTP
from scipy.optimize import minimize_scalar
# from GP13_UD_T3_offline import grand_T3_trigger, safe_substraction, get_DU_coord
plt.style.use("/Users/xishui/Dropbox/Config/presentation.mplstyle")

In [ ]:
coord_1078 = Geodetic(latitude=40.99368437530295, longitude=93.95411072589444, height=1205.9284000000027)
coord_DAQ = Geodetic(latitude= 40.99734117, longitude=93.94868278, height=1205.9284000000027)

def get_DU_coord(lat, long, alt, obstime, origin=coord_DAQ):
  # From GPS to Cartisian coordinates
  geod = Geodetic(latitude=lat, longitude=long, height=alt)
  gcs = GRANDCS(geod, obstime=obstime, location=origin)
  return gcs

# Merge coinc table from different CD

In [ ]:
file_coinctable = np.zeros((0, 4), dtype=np.float64)
file_ducoord = np.zeros((0, 4), dtype=np.float64)
file_rec_sphere = np.zeros((0, 9), dtype=np.float64)
file_rec_plane = np.zeros((0, 8), dtype=np.float64)
file_du_id = np.zeros((0,1), dtype=np.int16)
list_fname = np.zeros((0,1), dtype="U200")
list_du_sec0  = np.zeros((0,1), dtype=np.int64)
list_du_nano0 = np.zeros((0,1), dtype=np.int64)

i_event = 0
i_row = 0
list_dir = glob.glob("coincidence_table/daemon_job/*.root/")
list_dir = np.sort(list_dir)
for dir in list_dir:
    # Check the file size, skip the zeros
    if os.path.isfile(f"{dir}/Rec_plane_wave_recons.txt") == 0:
        continue
    _file_coinctable = np.genfromtxt(f"{dir}/Rec_coinctable.txt", dtype=float)
    _file_ducoord = np.genfromtxt(f"{dir}/coord_antennas.txt", dtype=float)
    _file_rec_sphere = np.genfromtxt(f"{dir}/Rec_sphere_wave_recons.txt", dtype=float, usecols=np.arange(9))
    _file_rec_sphere = _file_rec_sphere.reshape((-1,9))
    _file_rec_plane = np.genfromtxt(f"{dir}/Rec_plane_wave_recons.txt", dtype=float)
    _file_rec_plane = _file_rec_plane.reshape((-1,8))
    _file_du_id = np.genfromtxt(f"{dir}/DU_id.txt", usecols=1)
    _list_fname = np.genfromtxt(f"{dir}/DU_id.txt", usecols=0, dtype='U200')
    _list_du_sec0  = np.genfromtxt(f"{dir}/DU_id.txt", usecols=2, dtype=np.int64)
    _list_du_nano0 = np.genfromtxt(f"{dir}/DU_id.txt", usecols=3, dtype=np.int64)
    # Update the event number and row number
    _n_row = len(_file_coinctable[:,0])
    # Event ID starting from 0, so the event number = Max(ID) + 1
    _n_event = int(_file_coinctable[-1,1]) + 1
    _file_coinctable[:,0] = _file_coinctable[:,0] + i_row
    _file_coinctable[:,1] = _file_coinctable[:,1] + i_event
    _file_ducoord[:,0] = _file_ducoord[:,0] + i_row
    _file_rec_plane[:,0] = _file_rec_plane[:,0] + i_event
    _file_rec_sphere[:,0] = _file_rec_sphere[:,0] + i_event
    i_row += _n_row
    i_event += _n_event
    file_coinctable = np.append(file_coinctable, _file_coinctable, axis=0)
    file_ducoord = np.append(file_ducoord, _file_ducoord, axis=0)
    file_rec_sphere = np.append(file_rec_sphere, _file_rec_sphere, axis=0)
    file_rec_plane = np.append(file_rec_plane, _file_rec_plane, axis=0)
    file_du_id = np.append(file_du_id, _file_du_id)
    list_fname = np.append(list_fname, _list_fname)
    list_du_sec0  = np.append(list_du_sec0, _list_du_sec0)
    list_du_nano0 = np.append(list_du_nano0, _list_du_nano0)

# The distribution of $\chi^2$

## PWF

In [ ]:
list_chi2_PWF = file_rec_plane[:,6] 
plt.hist(list_chi2_PWF, np.logspace(-8, 8, 51), histtype='step', color='k')
# plt.hist(list_chi2[file_rec_plane[:,1] == 6], np.logspace(-2, 6, 51), histtype='step')
plt.semilogy()
plt.ylabel("# of events")
plt.xlabel("$\chi^2_{\\rm PWF}$")
plt.grid()
plt.semilogx()

## SWF

In [ ]:
list_chi2_SWF = file_rec_sphere[:,2]
plt.hist(list_chi2_SWF, np.logspace(-7, 9,), histtype='step')
plt.loglog()
# plt.legend()
plt.xticks()
plt.grid()
plt.xlabel(r"$\chi^2_{\rm SWF}$");

## Chi2 mask

In [ ]:
mask_chi2 = list_chi2_PWF < 1e3

# Event time

In [ ]:
# Get the true time (add the reference time back)
_, mask_event_time = np.unique(file_coinctable[:,1], return_index=True)
list_event_true = (file_coinctable[mask_event_time,2] * 1e9).astype(np.int64) + (list_du_sec0[mask_event_time] * 1e9).astype(np.int64) + list_du_nano0[mask_event_time]

### CD trigger rate

In [ ]:
t_bin_width = 2 * 1e9 # in nanosecond

t_start = list_event_true[0]
t_end = list_event_true[-1]
t_bin_edges = np.arange(t_start, t_end + t_bin_width, t_bin_width)
t_bin_centers = t_bin_edges[:-1] + t_bin_width // 2
n_CD, _ = np.histogram(list_event_true, t_bin_edges)
# plt.plot((t_bin_centers - t_bin_centers[0]) / 1e9,
#          n_CD / (t_bin_width / 1e9),
#          marker='.', ls='', markersize=1)
plt.plot(t_bin_centers[n_CD>0], n_CD[n_CD>0] / (t_bin_width / 1e9),
         marker='.', ls='-', markersize=3, lw=1)
plt.xlabel("Timestamp")
plt.ylabel("Trigger rate [Hz]")

## Time difference between two events

In [ ]:
plt.hist(np.diff(list_event_true[mask_chi2]), np.logspace(4, 15, 60))
plt.loglog()
plt.xlabel("$\Delta t [\\rm ns]$")
plt.ylabel("# of events");

# Reconstructed directions

## PWF

### Zenith

In [ ]:
list_n_du = file_rec_plane[:,1]
list_chi2 = file_rec_plane[:,6]

zenith = file_rec_plane[:,2].copy() # Propagation driection of the em wave: from the source to the observer
zenith = 180 - zenith # From the observer to the source
zenith[zenith > 90] = 90 - (zenith[zenith > 90] - 90) # Reflect the up-going events to down-going
plt.hist(zenith, bins=np.linspace(0, 90, 451), label='Total', histtype='step', lw=3, color='k')
plt.hist(zenith[list_n_du == 6], bins=np.linspace(0, 90, 451), label='n$_{DU}=6$', histtype='step', lw=1)
plt.hist(zenith[list_n_du == 7], bins=np.linspace(0, 90, 451), label='n$_{DU}=7$', histtype='step', lw=1)

plt.legend()
plt.xlabel('Zenith [deg] (wrapped at the horizon)')
plt.grid()
plt.semilogy()
plt.xlim(0, 90)
# plt.xticks(np.arange(0, 91, 10))
plt.tight_layout()
plt.legend(loc='best')
# plt.savefig("imgs/zenith_dist_beacon_173.pdf");

### Azimuth

In [ ]:
# change the coordinate system: "from source to observer" to "from observer to source"
# the conventional CR notation
azimuth = file_rec_plane[:,4] + 180
azimuth[azimuth > 360] = azimuth[azimuth > 360] - 360
plt.hist(azimuth, bins=np.linspace(0, 360, 361), label='Total', histtype='step', lw=1, color='k', zorder=100)
plt.legend()
plt.xlabel("Azimuth [deg]")
# plt.xlim(27, 28)
plt.grid()
plt.tight_layout()
# plt.savefig("imgs/azimuth_dist_beacon_173.pdf")

## SWF

In [ ]:
zenith_SWF = np.rad2deg(np.arccos( file_rec_sphere[:,6] / file_rec_sphere[:,8]))
zenith_SWF[zenith_SWF > 90] = 180 - zenith_SWF[zenith_SWF > 90]
azimuth_SWF = np.rad2deg(np.arctan2(file_rec_sphere[:,5], file_rec_sphere[:,4]))
azimuth_SWF[azimuth_SWF < 0] = azimuth_SWF[azimuth_SWF < 0] + 360
# plt.hist(_theta[list_chi2 < 1e3], np.linspace(0, 91, 901), histtype='step')
# plt.hist(_theta[list_chi2 > 1e4], np.linspace(0, 91, 901), histtype='step');
# plt.hist(_phi[~mask_chi2], np.linspace(0, 361, 3601), histtype='step')
# plt.hist(_phi[mask_chi2], np.linspace(0, 361, 3601), histtype='step');
plt.hist(zenith_SWF[mask_chi2], 200, histtype='step');
# plt.xlim(26, 28)

In [ ]:
plt.hist(azimuth_SWF[mask_chi2], np.linspace(0, 360, 361),
         histtype='step');

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(azimuth_SWF, zenith_SWF, marker='.', ls='', markersize=2, label='PWF')
plt.plot(azimuth, zenith, marker='.', ls='', markersize=2, label='SWF')
plt.ylim(90, 0)
plt.legend()

# Multiplicity distribution

In [ ]:
# n_DU, n = np.unique(file_rec_plane[~mask_chi2,1], return_counts=True)
# plt.bar(n_DU, n, width=0.7, alpha=0.4)
n_DU, n = np.unique(file_rec_sphere[mask_chi2,1], return_counts=True)
plt.bar(n_DU, n, width=0.7, alpha=1)
plt.plot(n_DU, n[0] * 10**(-0.65 * (n_DU - n_DU[0])), color='r')

plt.grid()
plt.xticks(np.arange(13))
plt.semilogy();

# Reconstructed source position

## (X, Y)

In [ ]:
x = file_rec_sphere[:,4]
y = file_rec_sphere[:,5]
z = file_rec_sphere[:,6]
d = np.sqrt((x - source[0])**2 + (y - source[1])**2 + (z - source[2]))
mask_d = d < (d.max() * 0.9)
plt.plot(-y[(mask_chi2) & (mask_d)], x[(mask_chi2) & (mask_d)], marker='.', ls='', markersize=2)
# plt.plot(y[list_chi2 < 1e3], -x[list_chi2 < 1e3], marker='.', ls='')
# plt.plot(y[list_chi2 > 1e4], -x[list_chi2 > 1e4], marker='.', ls='')
# plt.xlim(-100, 200)
# plt.ylim(-200, 200)
plt.xlabel("Easting [m]")
plt.ylabel("Northing [m]")
plt.axis("equal")

## Distance, is there a boundary?

In [ ]:
source = (0, 0, 1205)

In [ ]:
plt.hist(d[(mask_chi2) & mask_d], np.logspace(2, 5, 100), histtype='step')
# plt.hist(d[~mask_chi2], np.logspace(0, 5, 100), histtype='step')

plt.loglog()
plt.xlabel("Distance to (0,0, 1205) [m]");

## Error here, inconsistent number of events for sphere rec and time

In [ ]:
ax = plt.figure(figsize=(10, 10)).add_subplot(projection='3d')
# Flip the z values to all positive
ax.scatter(x[(mask_chi2) & (mask_d)], y[(mask_chi2) & (mask_d)], 
           np.abs(z[(mask_chi2) & (mask_d)] - source[2]) + - source[2],
           s=5, c=list_event_true[(mask_chi2) & (mask_d)])
ax.scatter(*source, marker='*', s=500,)
ax.view_init(elev=30., azim=230, roll=0)
ax.set_xlabel("x[m]")
ax.set_ylabel("y[m]")
ax.set_zlabel("z[m]")
ax.legend()
plt.tight_layout()